%pip install --upgrade boto3

In [1]:
import sagemaker
import boto3
import pandas as pd
import urllib
import numpy as np
import os

from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

bucketName = 'bktsagenote' 
my_region = boto3.session.Session().region_name

s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucketName)
    print('S3 bucket object made')
except Exception as e:
    print('S3 error: ',e)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
S3 bucket object made


In [2]:
prefix = 'xgboost-build'
output_path ='s3://{}/{}/output'.format(bucketName, prefix)
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-\
        train-deploy-machine-learning-model-sagemaker/bank_clean.\
        27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
except Exception as e:
    print('data error: ',e)
try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('data read.')
except Exception as e:
    print('data error: ',e)

data error:  URL can't contain control characters. '/tmt/build-        train-deploy-machine-learning-model-sagemaker/bank_clean.        27f01fbbdf43271788427f3682996ae29ceca05d.csv' (found at least ' ')
data read.


In [3]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data))])

pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], 
    axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketName).Object(os.path.join(prefix, 
    'train/train.csv')).upload_file('train.csv')
s3_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(
    bucketName, prefix), content_type='csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], 
    axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketName).Object(os.path.join(prefix, 
    'train/train.csv')).upload_file('train.csv')
s3_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(
    bucketName, prefix), content_type='csv')
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], 
    axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketName).Object(os.path.join(prefix, 
    'test/test.csv')).upload_file('test.csv')
s3_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(
    bucketName, prefix), content_type='csv')

Model building

In [6]:
estimator.fit({'train': s3_train,'validation': s3_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-05-29-02-35-50-403


2024-05-29 02:35:50 Starting - Starting the training job...
2024-05-29 02:36:05 Starting - Preparing the instances for training...
2024-05-29 02:36:44 Downloading - Downloading input data...
2024-05-29 02:36:59 Downloading - Downloading the training image...
2024-05-29 02:37:24 Training - Training image download completed. Training in progress..[2024-05-29 02:37:46.148 ip-10-2-64-139.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined deli